RESNET, nfolds=5, 50-15-15, patience=5, resize=96, DataAugment= zoom, optimizers.Adam, activation=sigmoid

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from subprocess import check_output
#print(check_output(["ls", ".."]).decode("utf8"))

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
from tqdm import tqdm

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

from sklearn.cross_validation import KFold
from sklearn.metrics import fbeta_score
import time

Using TensorFlow backend.
/home/eric/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
x_train = []
x_test = []
y_train = []

df_train = pd.read_csv('train_v2.csv')
df_test = pd.read_csv('sample_submission_v2.csv')

In [3]:
flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

In [4]:
labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'cultivation',
 'artisinal_mine',
 'haze',
 'primary',
 'slash_burn',
 'habitation',
 'clear',
 'road',
 'selective_logging',
 'partly_cloudy',
 'agriculture',
 'water',
 'cloudy']

In [5]:
label_map = {'agriculture': 14,
 'artisinal_mine': 5,
 'bare_ground': 1,
 'blooming': 3,
 'blow_down': 0,
 'clear': 10,
 'cloudy': 16,
 'conventional_mine': 2,
 'cultivation': 4,
 'habitation': 9,
 'haze': 6,
 'partly_cloudy': 13,
 'primary': 7,
 'road': 11,
 'selective_logging': 12,
 'slash_burn': 8,
 'water': 15}

In [6]:
for f, tags in tqdm(df_train.values, miniters=1000):
    img = cv2.imread('./train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, (96, 96)))
    y_train.append(targets)

100%|██████████| 40479/40479 [01:00<00:00, 668.30it/s]


In [7]:
x_train = np.array(x_train, np.float32)/255.

In [8]:
y_train = np.array(y_train, np.uint8)

In [9]:
for f, tags in tqdm(df_test.values, miniters=1000):
    img = cv2.imread('./test-jpg/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (96, 96)))

100%|██████████| 61191/61191 [01:31<00:00, 668.13it/s]


In [10]:
x_test  = np.array(x_test, np.float32)/255.

In [11]:
print(x_train.shape)
print(y_train.shape)

(40479, 96, 96, 3)
(40479, 17)


In [17]:
nfolds = 5

num_fold = 0
sum_score = 0

yfull_test = []
yfull_train =[]

In [13]:
batch_size = 128

In [14]:
kf = KFold(len(y_train), n_folds=nfolds, shuffle=True, random_state=1)

In [15]:
datagen = ImageDataGenerator(zoom_range=0.2, horizontal_flip=True, vertical_flip=True)

In [18]:
for train_index, test_index in kf:
        start_time_model_fitting = time.time()
        
        X_train = x_train[train_index]
        Y_train = y_train[train_index]
        X_valid = x_train[test_index]
        Y_valid = y_train[test_index]

        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        print('Split train: ', len(X_train), len(Y_train))
        print('Split valid: ', len(X_valid), len(Y_valid))
        
        kfold_weights_path = os.path.join('', 'weights_kfold_' + str(num_fold) + '.h5')
        
        model = Sequential()
        model.add(BatchNormalization(input_shape=(96, 96,3)))
        model.add(Conv2D(32, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(256, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(256, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
             
        model.add(Conv2D(512, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(512, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(512, kernel_size=(3, 3),padding='same', activation='relu'))
        model.add(Conv2D(1024, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        
        model.add(Flatten())
        model.add(Dense(1024, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(17, activation='sigmoid'))

        epochs_arr = [50, 15, 15]
        learn_rates = [0.001, 0.0001, 0.00001]

        for learn_rate, epochs in zip(learn_rates, epochs_arr):
            opt  = optimizers.Adam(lr=learn_rate)
            model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                          optimizer=opt,
                          metrics=['accuracy'])
            callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=0),
            ModelCheckpoint(kfold_weights_path, monitor='val_loss', save_best_only=True, verbose=0)]

            #model.fit(x = X_train, y= Y_train, validation_data=(X_valid, Y_valid),
                  #batch_size=128,verbose=2, epochs=epochs,callbacks=callbacks,shuffle=True)
            
            model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                            validation_data=datagen.flow(X_valid, Y_valid, batch_size=batch_size),
                            validation_steps=len(X_valid) / batch_size,
                            steps_per_epoch=len(X_train) / batch_size,
                            epochs=epochs,
                            callbacks=callbacks,
                            workers=4)
        
        if os.path.isfile(kfold_weights_path):
            model.load_weights(kfold_weights_path)
        
        p_valid = model.predict(X_valid, batch_size = batch_size, verbose=2)
        print(fbeta_score(Y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

        p_train = model.predict(x_train, batch_size = batch_size, verbose=2)
        yfull_train.append(p_train)
        
        p_test = model.predict(x_test, batch_size = batch_size, verbose=2)
        yfull_test.append(p_test)

Start KFold number 1 from 2
Split train:  32383 32383
Split valid:  8096 8096
Epoch 1/50
253/252 [==============================] - 114s - loss: 0.2511 - acc: 0.9085 - val_loss: 0.2474 - val_acc: 0.9071
Epoch 2/50
253/252 [==============================] - 117s - loss: 0.1577 - acc: 0.9377 - val_loss: 0.1707 - val_acc: 0.9344
Epoch 3/50
253/252 [==============================] - 117s - loss: 0.1475 - acc: 0.9418 - val_loss: 0.1402 - val_acc: 0.9450
Epoch 4/50
253/252 [==============================] - 116s - loss: 0.1406 - acc: 0.9447 - val_loss: 0.1355 - val_acc: 0.9483
Epoch 5/50
253/252 [==============================] - 117s - loss: 0.1345 - acc: 0.9473 - val_loss: 0.1341 - val_acc: 0.9480
Epoch 6/50
253/252 [==============================] - 115s - loss: 0.1296 - acc: 0.9497 - val_loss: 0.1478 - val_acc: 0.9461
Epoch 7/50
253/252 [==============================] - 115s - loss: 0.1268 - acc: 0.9504 - val_loss: 0.1367 - val_acc: 0.9494
Epoch 8/50
253/252 [===========================

253/252 [==============================] - 118s - loss: 0.1647 - acc: 0.9339 - val_loss: 0.1703 - val_acc: 0.9308
Epoch 4/50
253/252 [==============================] - 119s - loss: 0.1574 - acc: 0.9368 - val_loss: 0.1622 - val_acc: 0.9355
Epoch 5/50
253/252 [==============================] - 118s - loss: 0.1467 - acc: 0.9412 - val_loss: 0.1404 - val_acc: 0.9429
Epoch 6/50
253/252 [==============================] - 120s - loss: 0.1400 - acc: 0.9440 - val_loss: 0.1355 - val_acc: 0.9470
Epoch 7/50
253/252 [==============================] - 117s - loss: 0.1364 - acc: 0.9463 - val_loss: 0.1478 - val_acc: 0.9436
Epoch 8/50
253/252 [==============================] - 119s - loss: 0.1328 - acc: 0.9481 - val_loss: 0.1245 - val_acc: 0.9523
Epoch 9/50
253/252 [==============================] - 117s - loss: 0.1278 - acc: 0.9499 - val_loss: 0.1323 - val_acc: 0.9499
Epoch 10/50
253/252 [==============================] - 119s - loss: 0.1250 - acc: 0.9513 - val_loss: 0.1243 - val_acc: 0.9504
Epoch 11/5

KeyboardInterrupt: 

In [19]:
result = np.array(yfull_test[0])
for i in range(1, nfolds):
    result += np.array(yfull_test[i])
result /= nfolds
result = pd.DataFrame(result, columns = labels)
result

,blow_down,bare_ground,conventional_mine,blooming,cultivation,artisinal_mine,haze,primary,slash_burn,habitation,clear,road,selective_logging,partly_cloudy,agriculture,water,cloudy
0,2.696447e-04,0.000358,2.406719e-05,0.006320,0.002378,3.936566e-05,0.003992,0.999663,0.000049,0.002259,0.993404,0.003125,0.000851,0.001140,0.006522,0.004196,1.430247e-04
1,8.865900e-04,0.000307,2.516026e-05,0.017380,0.004920,5.138692e-05,0.001310,0.999611,0.000078,0.003413,0.988229,0.002605,0.001984,0.010628,0.012013,0.005408,2.138614e-04
2,1.489264e-05,0.000061,3.589628e-06,0.000008,0.004887,5.694415e-07,0.000039,0.999553,0.000064,0.000748,0.000400,0.005297,0.000007,0.998531,0.012264,0.036455,2.765786e-04
3,9.049254e-03,0.007427,1.754835e-04,0.032438,0.287778,2.303507e-04,0.008498,0.999160,0.004483,0.006651,0.857192,0.017480,0.015993,0.151342,0.400219,0.026378,9.126219e-04
4,3.017369e-05,0.000129,7.303674e-06,0.000039,0.003284,8.314435e-06,0.005580,0.739574,0.000036,0.001106,0.002813,0.007777,0.000051,0.729270,0.014377,0.030751,2.682497e-01
5,2.116224e-04,0.000117,7.282159e-06,0.000250,0.001313,7.089475e-06,0.000235,0.999929,0.000027,0.000673,0.999242,0.001130,0.000086,0.000566,0.003536,0.003023,2.557572e-05
6,2.681561e-04,0.017438,9.201771e-04,0.002622,0.535426,3.565052e-04,0.272234,0.998971,0.010898,0.889062,0.213085,0.119661,0.003390,0.506714,0.669601,0.046895,4.614639e-04
7,1.617710e-05,0.033432,9.706481e-05,0.000024,0.005142,4.180264e-05,0.008801,0.565448,0.000031,0.996331,0.984489,0.995254,0.000106,0.004194,0.239047,0.052621,4.499381e-04
8,4.329217e-04,0.000307,3.048430e-05,0.000196,0.001775,4.373241e-05,0.000542,0.999669,0.000043,0.000926,0.998490,0.002120,0.000111,0.000923,0.004972,0.005863,1.412200e-04
9,2.093076e-04,0.029276,2.344564e-04,0.000253,0.272910,8.406038e-05,0.766305,0.995795,0.005197,0.010137,0.230813,0.046950,0.000337,0.008951,0.835578,0.055082,1.238573e-03


In [20]:
from tqdm import tqdm
thres = [0.07, 0.17, 0.2, 0.04, 0.23, 0.33, 0.24, 0.22, 0.1, 0.19, 0.23, 0.24, 0.12, 0.14, 0.25, 0.26, 0.16]
preds = []
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.ix[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

100%|██████████| 61191/61191 [01:40<00:00, 610.22it/s]


In [21]:
p_valid = model.predict(X_valid, batch_size = 128, verbose=2)
print(fbeta_score(Y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

0.510115202891


In [22]:
# Let's save those weights.
model.save_weights('Kele_XU_Keras_0913-Copy9-ter-ResNet50-96.h5')

In [23]:
df_test['tags'] = preds
df_test.to_csv('submission_keras_5_fold_CV_0.9136_LB_0.913_copy9-ter-RestNet50-96.csv', index=False)